<a href="https://colab.research.google.com/github/LEAF-BoiseState/Idaho-Mesonet-Design/blob/main/Idaho_SnowCover_Clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
try:
    import wxee
except ImportError:
    print('wxee package not installed. Installing...')
    subprocess.check_call(['python','-m','pip','install','wxee'])
  #!pip install eemont

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from google.colab import drive
drive.mount('/content/gdrive')

import wxee

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
snow_write_path = '_Idaho_TopoVeg/'

idaho_annmax_sca_nc = 'snowcover_ID_annmax.nc'


In [29]:
# Get the USGS HUC08 collection from the data catalog
us_states_tiger = ee.FeatureCollection('TIGER/2018/States')

id_outline = us_states_tiger.filter(ee.Filter.inList('STATEFP', ee.List(['16'])))

Map = emap.Map(center=(47.5, -116.25),zoom=9)

Map.addLayer(ee.Image().paint(id_outline, 0, 2), {}, 'Idaho')

Map.addLayerControl()
Map

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [41]:
#ee.Filter.date('2000-10-01','2023-09-30')

#mod10a1_idaho = ee.ImageCollection("MODIS/061/MOD10A1").filter().map(lambda image: image.clip(id_outline))

def get_annual_snow_sequence(year):
    date_start = ee.Date.fromYMD(year,10,1)
    date_end = date_start.advance(1,'year')

    return(ee.ImageCollection("MODIS/061/MOD10A1")
      .select('NDSI_Snow_Cover')
      .filterDate(date_start, date_end)
      .map(lambda image: image.where(image.gt(0).And(image.lte(100)), 1))
      .map(lambda image: image.where(image.lte(0).Or(image.gt(100)), 0))
      .sum()
      .set({'year': year, 'system:time_start':date_start.millis()})
      .set({'year': year, 'system:time_end':date_end.millis()})
      .clip(id_outline))


id_annmax_fsca = ee.ImageCollection(ee.List.sequence(2000, 2022).map(lambda year: get_annual_snow_sequence(year)))


In [42]:
Map2 = emap.Map(center=(47.5, -116.25),zoom=5)

fscaVis = {
  'min': 0.0,
  'max': 100.0,
  'gamma': 2.0,
}

Map2.addLayer(id_annmax_fsca, fscaVis, 'Annual Maximum fSCA')
Map2.addLayerControl()
Map2

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [45]:
snow_cover_days = 50

id_seasonal_snow_covered = id_annmax_fsca.map(lambda image: image.where(image.lte(snow_cover_days),0).where(image.gt(snow_cover_days),1)).sum()

id_snow_covered_days = id_annmax_fsca.mean()

Map3 = emap.Map(center=(47.5, -116.25),zoom=5)

ssdVis = {
  'min': 0.0,
  'max': 23.0,
  'gamma': 2.0,
}

ascdVis = {
  'min': 0.0,
  'max': 365.0,
  'gamma': 2.0,
}


Map3.addLayer(id_seasonal_snow_covered, ssdVis, 'Number of Years With > 49 days snow covered')
#Map3.addLayer(id_snow_covered_days, ascdVis, 'Average Annual Number of Snow Covered Days')
Map3.addLayerControl()
Map3

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [46]:
id_seasonal_snow_covered = id_seasonal_snow_covered.toDouble().unmask(-9999)
id_seasonal_snow_covered = id_seasonal_snow_covered.clip(id_outline.geometry().bounds())

id_snow_covered_days = id_snow_covered_days.toDouble().unmask(-9999)
id_snow_covered_days = id_snow_covered_days.clip(id_outline.geometry().bounds())


downConfig = {'scale': 500, "maxPixels": 1.0E13, 'driveFolder': '_Idaho_TopoVeg' }  # scale means resolution.

task1 = ee.batch.Export.image(id_seasonal_snow_covered, 'ID_seasonally_snow_covered_'+str(snow_cover_days)+'days', downConfig)
task1.start()
print(task1.status())

task2 = ee.batch.Export.image(id_snow_covered_days, 'ID_mean_snow_covered_days_'+str(snow_cover_days)+'days', downConfig)
task2.start()
print(task2.status())



{'state': 'READY', 'description': 'ID_seasonally_snow_covered_50days', 'creation_timestamp_ms': 1697476750872, 'update_timestamp_ms': 1697476750872, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '6RBYB3VC6B4LG7NRFXU5IRMD', 'name': 'projects/earthengine-legacy/operations/6RBYB3VC6B4LG7NRFXU5IRMD'}
{'state': 'READY', 'description': 'ID_mean_snow_covered_days_50days', 'creation_timestamp_ms': 1697476751710, 'update_timestamp_ms': 1697476751710, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '76WGN3FAZI762E2XN47QC53E', 'name': 'projects/earthengine-legacy/operations/76WGN3FAZI762E2XN47QC53E'}
